In [ ]:
import pandas as pd
import numpy as np 
import glob 
import warnings 
import plotly.express as px 
import seaborn as sns 
import matplotlib.pyplot as plt 
import math
import os 

In [ ]:
def wap1(row):
    denom = row.ask_size1 + row.bid_size1
    return ((row.bid_price1 * row.ask_size1 + row.ask_price1 * row.bid_size1)/denom)
    
def wap2(row):
    denom = row.ask_size2 + row.bid_size2
    return ((row.bid_price2 * row.ask_size2 + row.ask_price2 * row.bid_size2)/denom)

def log_avg_wap(row):
    return np.log((row.wap1 + row.wap2)/2)

def log_return(list_prices):
    return np.log(list_prices).diff()
def realized_volatility(series_log_return):
    return np.sqrt(np.sum(series_log_return**2))
def custom_loss(ytrue,ypred) :
    squared_residual = (ytrue-ypred)**2/ytrue
    grad = squared_residual
    hess = np.ones(len(ytrue))
    
    return grad,hess

def rmspe(y_true, y_pred):
    return  (np.sqrt(np.mean(np.square((y_true - y_pred) / y_true))))
def feval_RMSPE(preds, train_data):
    labels = train_data.get_label()
    return 'RMSPE', round(rmspe(y_true = labels, y_pred = preds),5), False
def custom_rmspe_valid(y_true, y_pred):
    residual = (y_true - y_pred).astype("float")
    residual = residual ** 2 / y_true
    residual = np.mean(residual)
    return "eval_RMSPE", math.sqrt(residual), False
def simple_volatility(series_prix):
    mx = np.max(series_prix)
    mn = np.min(series_prix)
    moy = np.mean(series_prix)
    vol = (moy-mn)/(mx-mn)
    return vol
def count_unique(series):
    return len(np.unique(series))

## 1. Importing the Data


In [ ]:

train = pd.read_parquet("target_data/target_train.parquet")
train

In [ ]:
book_train_stock_id_0 = pd.read_parquet("stock_book_train/stock_18_train.parquet")
book_train_stock_id_0_t_id5 = book_train_stock_id_0[book_train_stock_id_0["time_id"]==5]
#book_train_stock_id_0_t_id5
book_train_stock_id_0_t_id5

In [ ]:
trade_train_stock_id_0 = pd.read_parquet("stock_trade_train/stock_18_train.parquet")
trade_train_stock_id_0

In [ ]:
trade_train_stock_id_0.groupby(['time_id']).mean().reset_index().iloc[1013:1015,]

## 2 i) Feature Engineering (TRADE DATA)


In [ ]:
#Feature 1 -> Relative volume by 10min bucket

#1. Finding average size (volume) across all time_id's

avg_trade_volume_stock_id_0 = trade_train_stock_id_0.groupby(['time_id']).sum()["size"].mean() #this groups by time id, gets the total sum of the size, then it gets the mean across all time buckets 
 
#2. Get size (volume) of time_id = 5
trade_volume_stock_id_0_time_id_5 = trade_train_stock_id_0[trade_train_stock_id_0['time_id'] == 5]["size"].sum() #gives me the volume of time bucket 5

#3. Compute relative volume
rel_trade_volume_stock_id_0_time_id_5 = trade_volume_stock_id_0_time_id_5/avg_trade_volume_stock_id_0
rel_trade_volume_stock_id_0_time_id_5

In [ ]:
avg_trade_volume_stock_id_0

In [ ]:
trade_train_stock_id_0['time_id'].mean()

In [ ]:
#Feature 2 -> Relative price range 

#1. Find min and max of trade price for all seperate time_id. 
min_trade_price_stock_id_0 = trade_train_stock_id_0.groupby(['time_id']).min()
max_trade_price_stock_id_0 = trade_train_stock_id_0.groupby(['time_id']).max()
#2. Find range of trade price and median of trade price for all seperate time_id. 
range_trade_price_stock_id_0 = max_trade_price_stock_id_0 - min_trade_price_stock_id_0 
median_trade_price_stock_id_0 = trade_train_stock_id_0.groupby(['time_id']).median()
#3. Use median to compute how much percent below our minimum is for all seperate time_id. 
lower_percent_range_relative_to_median = (median_trade_price_stock_id_0 - min_trade_price_stock_id_0)/median_trade_price_stock_id_0
#4. Use median to compute how much percent above our maximum is for all seperate time_id. 
upper_percent_range_relative_to_median = (max_trade_price_stock_id_0 - median_trade_price_stock_id_0)/median_trade_price_stock_id_0
#5 Add both values to get total percent range. E.g. 3% below median and 5% above median = 8% total range for all seperate time_id.
total_percent_range = upper_percent_range_relative_to_median + lower_percent_range_relative_to_median

#6. Compute the average percent range across all time id's

avg_total_percent_range = total_percent_range["price"].mean()

#7. Get total percent range for time id 5
min_price_stock_id_0_time_id_5 = trade_train_stock_id_0[trade_train_stock_id_0['time_id'] == 5]["price"].min()
max_price_stock_id_0_time_id_5 = trade_train_stock_id_0[trade_train_stock_id_0['time_id'] == 5]["price"].max()

range_stock_id_0_time_id_5 = max_price_stock_id_0_time_id_5 - min_price_stock_id_0_time_id_5 #gives me the range of price for time bucket 5
median_stock_id_0_time_id_5 = trade_train_stock_id_0[trade_train_stock_id_0['time_id'] == 5]["price"].median()

lower_percent_range_relative_to_median_stock_id_0_time_id_5 = (median_stock_id_0_time_id_5 - min_price_stock_id_0_time_id_5)/median_stock_id_0_time_id_5

upper_percent_range_relative_to_median_stock_id_0_time_id_5= (max_price_stock_id_0_time_id_5 - median_stock_id_0_time_id_5)/median_stock_id_0_time_id_5

total_percent_range_stock_id_0_time_id_5 = lower_percent_range_relative_to_median_stock_id_0_time_id_5 + upper_percent_range_relative_to_median_stock_id_0_time_id_5

#8. Compute relative percent trading range to average
rel_total_percent_range_stock_id_0_time_id_5 = total_percent_range_stock_id_0_time_id_5/avg_total_percent_range
rel_total_percent_range_stock_id_0_time_id_5

In [ ]:
total_percent_range

In [ ]:
avg_total_percent_range

In [ ]:
total_percent_range

In [ ]:
#Feature 3 -> size_per_order
size = 
size_per_order = trade_train_stock_id_0.groupby(['time_id']).sum()['order_count']
order_count

In [ ]:
#Feature 3 -> Time between execution --> size/second 

## 2 ii) Feature Engineering (BOOK DATA)


In [ ]:
###Feature Engineering for ORDER BOOK data a###
book_train_stock_id_0.groupby(['time_id']).mean()

In [ ]:
# Compute the first weighted averaged price for each seconds_in_bucket and time ID
 
#book_train_stock_id_0["wap1"] = book_train_stock_id_0.apply(wap1,axis=1)
denom1 = book_train_stock_id_0["ask_size1"] + book_train_stock_id_0["bid_size1"]
volprice1 = book_train_stock_id_0["bid_price1"] * book_train_stock_id_0["ask_size1"] + book_train_stock_id_0["ask_price1"] * book_train_stock_id_0["bid_size1"]
book_train_stock_id_0["wap1"] = volprice1/denom1

In [ ]:
# Compute the second weighted averaged price for each seconds_in_bucket and time ID 

#book_train_stock_id_0.loc[:, "wap2"] = book_train_stock_id_0.apply(wap2,axis=1)
denom2 = book_train_stock_id_0["ask_size2"] + book_train_stock_id_0["bid_size2"]
volprice2 = book_train_stock_id_0["bid_price2"] * book_train_stock_id_0["ask_size2"] + book_train_stock_id_0["ask_price2"] * book_train_stock_id_0["bid_size2"]
book_train_stock_id_0["wap2"] = volprice2/denom2

In [ ]:
# Compute the avg weighted price using both wap1 and wap2 for each seconds_in_bucket and time ID 

#book_train_stock_id_0.loc[:, "log_avg_wap"] = book_train_stock_id_0.apply(log_avg_wap,axis=1)
book_train_stock_id_0["avg_wap"] = (book_train_stock_id_0["wap1"] + book_train_stock_id_0["wap2"])/2

In [ ]:
#Theory, the bigger the spread the higher the volatility!!!
#Getting spread ratio's of 1 and 2
spread_ratio_1 = book_train_stock_id_0["ask_price1"]/book_train_stock_id_0["bid_price1"]
book_train_stock_id_0["spread_ratio_1"] = spread_ratio_1
spread_ratio_2 = book_train_stock_id_0["ask_price2"]/book_train_stock_id_0["bid_price2"]
book_train_stock_id_0["spread_ratio_2"] = spread_ratio_2

In [ ]:
#Compute volume imbalance as an average ratio (supply/demand) per time_id
total_bid_size = book_train_stock_id_0["bid_size1"] + book_train_stock_id_0["bid_size2"]
total_ask_size = book_train_stock_id_0["ask_size1"] + book_train_stock_id_0["ask_size2"]
book_train_stock_id_0["vol_imbalance"] = total_ask_size/total_bid_size
#Finding the average volume imbalance by time_id
vol_imbalance = book_train_stock_id_0.groupby(['time_id']).mean()["vol_imbalance"]
book_train_stock_id_0.groupby(['time_id']).mean()

#how much it deviates from a ratio of 1:1 means its more significant a ratio of 10 is the same significance as a ratio of 0.1? So maybe we standardize it and say if ratio <1 do 1/ratio... so ratios like 0.1 can be changed to 10!!!


## 3 Data Exploration (e.g. correlation)


In [ ]:
#Exploring previous realized vol against target next 10 min for stock id 0
target_train = pd.read_parquet("target_data/target_train.parquet")
target_train_stock_id_0 = target_train[target_train["stock_id"] == 0]
target_train_stock_id_0

In [ ]:
#get the log returns for stock 0 using avg_wap
book_train_stock_id_0.loc[:, 'log_return'] = log_return(book_train_stock_id_0["avg_wap"])
#just extract all rows that aren't null...
book_train_stock_id_0 = book_train_stock_id_0[~book_train_stock_id_0['log_return'].isnull()]
#examine time ID 5 by itself
book_train_stock_id_0_time_id_5 = book_train_stock_id_0[book_train_stock_id_0['time_id']==5]
book_train_stock_id_0_time_id_5

In [ ]:
#Plotting log returns for stock 0
fig = px.line(book_train_stock_id_0_time_id_5, x="seconds_in_bucket", y="log_return", title='Log return of stock_id_0, time_id_5')
fig.show()

In [ ]:
#Calculating realized volatility for stock0, time_id 5 
realized_vol = realized_volatility(book_train_stock_id_0_time_id_5['log_return'])
realized_vol

In [ ]:
#Is previous 10min volatility related to next 10 min vol???!!!
realized_vol_id_0 = book_train_stock_id_0.groupby("time_id")["log_return"].apply(realized_volatility) #going through book data of stock 0, grouping by time_id and getting realized volatility using log returns
realized_vol_id_0
vol_id_0_df = pd.DataFrame(columns=[target_train_stock_id_0["target"], realized_vol_id_0])
realized_vol_id_0
#target_train_stock_id_0["target"]
vol_id_0_df = vol_id_0_df.T
vol_id_0_df
vol_id_0_df.reset_index(inplace=True)
vol_id_0_df = vol_id_0_df.rename(columns={"target": "Target Volatility", "log_return": "Realized Volatility"})
vol_id_0_df

In [ ]:
fig = px.scatter(vol_id_0_df, x='Realized Volatility', y='Target Volatility', title='Target Volatility vs Realized Volatility Last 10 min')
fig.update_yaxes(nticks=20)
fig.update_xaxes(nticks=20)
fig.update_layout(xaxis_range=[0, 0.03])

fig.show()

In [ ]:
vol_id_0_df.corr()
#It appears that we have a strong positive correlation between realized volatility and target volatility of the prev 10 min? Is this random? Lets check the other 10 min intervals!!!!!
#WAP_AVG to calculate realized vol of first 10 mins
#WAP1 might be better? 

In [ ]:
#Conduct a quick random algorithm to show that only the current time_id's was related to the next 10 mins (i.e. target volatility), and previous time_id's had no effect on later time_id's... 

#Use sample function to shift around target volatility data randomly
vol_id_0_df_shifted = vol_id_0_df.copy()
vol_id_0_df_shifted["Target Volatility"] = vol_id_0_df_shifted["Target Volatility"].sample(frac=1).reset_index(drop=True)
#np.random.uniform(low=0, high=)
vol_id_0_df_shifted

In [ ]:
global test
global correlation_dict
correlation_dict = {
    "stock_id": [],
    "realized_t_corr": [],
    "realized_t_1_corr": [],
    "realized_t_2_corr": [],
    "realized_t_3_corr": [],
    "realized_t_4_corr": [],
    "realized_t_5_corr": [],

}
def isValidStock(i):
    filename = "stock_trade_train/stock_" + str(i) + "_train.parquet"
    print(filename)
    if not os.path.exists(filename):
        return False
    return True
global tester 
def corrHeatmap(subset):
    global correlation_dict
    for j in range(127):
        if not isValidStock(j):
            continue

        #generalizing an algorithm for all stocks to create average correlation matrix... 
        #Begin by reading in the target train data for every stock... 
        parquet_path = "stock_book_train/stock_" + str(j) + "_train.parquet"
        book_train_stock_id_j = pd.read_parquet(parquet_path)
        
        target_train = pd.read_parquet("target_data/target_train.parquet")
        #Filter out the stock we are interested in
        target_train_stock_id_j = target_train[target_train["stock_id"] == j]

        #getting wap
        # Compute the first weighted averaged price for each seconds_in_bucket and time ID 
        denom1 = book_train_stock_id_j["ask_size1"] + book_train_stock_id_j["bid_size1"]
        volprice1 = book_train_stock_id_j["bid_price1"] * book_train_stock_id_j["ask_size1"] + book_train_stock_id_j["ask_price1"] * book_train_stock_id_j["bid_size1"]
        book_train_stock_id_j["wap1"] = volprice1/denom1

        # Compute the second weighted averaged price for each seconds_in_bucket and time ID 

        denom2 = book_train_stock_id_j["ask_size2"] + book_train_stock_id_j["bid_size2"]
        volprice2 = book_train_stock_id_j["bid_price2"] * book_train_stock_id_j["ask_size2"] + book_train_stock_id_j["ask_price2"] * book_train_stock_id_j["bid_size2"]
        
        book_train_stock_id_j["wap2"] = volprice2/denom2
        book_train_stock_id_j["avg_wap"] = (book_train_stock_id_j["wap1"] + book_train_stock_id_j["wap2"])/2

        #get the log returns for stock 0 using avg_wap
        book_train_stock_id_j.loc[:, 'log_return'] = log_return(book_train_stock_id_j["avg_wap"])
        #just extract all rows that aren't null...
        test = book_train_stock_id_j
        book_train_stock_id_j = book_train_stock_id_j[~book_train_stock_id_j['log_return'].isnull()]
        
        #Is previous 10min volatility related to next 10 min vol???!!!
        book_train_stock_id_j = book_train_stock_id_j[book_train_stock_id_j["seconds_in_bucket"] >= subset] #Only consider seconds_in_bucket > subset 
        
        realized_vol_id_j = book_train_stock_id_j.groupby("time_id")["log_return"].apply(realized_volatility) #going through book data of stock 0, grouping by time_id and getting realized volatility using log returns-

        vol_id_j_df = pd.DataFrame(columns=[target_train_stock_id_j["target"], realized_vol_id_j]) #create a dataframe with target data and realized volatility for each time id
        vol_id_j_df = vol_id_j_df.T
        vol_id_j_df.reset_index(inplace=True)
        vol_id_j_df = vol_id_j_df.rename(columns={"target": "Target Volatility", "log_return": "Realized Volatility"})
        vol_id_j_df


        #get correlation of realized volatility of current time id with subsequent 10 min target volatility.  
        realized_t_corr = vol_id_j_df.corr().iloc[0,1]
        correlation_dict["realized_t_corr"].append(realized_t_corr)


        #Creating a copy of realized-volatiliy~target so we can forward shift 5 times and get the new correlation of every shift...
        vol_id_j_df_shifted = vol_id_j_df.copy()
        #print(j)
        for i in range(1,6):
            realized_t_i_corr = "realized_t_" + str(i) + "_corr"
            vol_id_j_df_shifted["Realized Volatility"] = vol_id_j_df_shifted["Realized Volatility"].shift(1)
            realized_t_i_corr_value = vol_id_j_df_shifted.corr().iloc[0,1]
            correlation_dict[realized_t_i_corr].append(realized_t_i_corr_value)

        correlation_dict["stock_id"].append(j)

corrHeatmap(0)

In [ ]:
global correlation_dict
average_correlation = {key: np.mean(value) for key,value in correlation_dict.items()}
average_correlation = pd.DataFrame(list(average_correlation.items()))
average_correlation = average_correlation.drop([0], axis=0)
col_names = average_correlation.iloc[:, 0]
average_correlation = average_correlation.drop([0], axis=1)
average_correlation.columns = ['Target']
average_correlation = average_correlation.set_index(col_names)
average_correlation.index.name = None


In [ ]:

fig = plt.figure(figsize=(10,10), dpi=200)
sns.heatmap(data=average_correlation.T,annot=True,square=True, fmt='f')
#ax.invert_yaxis()
fig.suptitle('Average Correlation between Realized Volatility (Last 10 min) and Target Volatility', fontsize=16)
plt.show()


In [ ]:
def wap(df):
        return (df['bid_price1'] * df['ask_size1'] +
                df['ask_price1'] * df['bid_size1']) / (df['bid_size1'] + df['ask_size1'])

def wap2(df):
    return (df['bid_price2'] * df['ask_size2'] +
            df['ask_price2'] * df['bid_size2']) / (df['bid_size2'] + df['ask_size2'])

def log_return(list_stock_prices):
    return np.log(list_stock_prices).diff()

def realized_volatility(returns):
    return np.sqrt(np.sum(returns ** 2))

def count_unique(series):
    return len(np.unique(series))

def isValidStock(i):
    filename = "stock_trade_train/stock_" + str(i) + "_train.parquet"
    print(filename)
    if not os.path.exists(filename):
        return False
    return True

In [ ]:
def book_predictors(stock_id, train_or_test):
    stock_data = pd.read_parquet('stock_book_' + train_or_test + '/stock_' + str(stock_id) + '_' + train_or_test + '.parquet')
    stock_data = stock_data[stock_data["seconds_in_bucket"] >= 300] 
    stock_data['avg_wap']= (wap(stock_data)+ wap2(stock_data))/2
    stock_data['log_return'] = stock_data.groupby('time_id')['avg_wap'].apply(log_return)

    create_feature_dict = {
            'log_return':[realized_volatility],
    }

    result = pd.DataFrame(stock_data.groupby(['time_id']).agg(create_feature_dict)).reset_index()
    result.columns = result.columns.map('_'.join).str.strip('_')
    return result


def target(stock_id, train_or_test):
    result = pd.read_parquet('target_data/target_' + train_or_test + '.parquet')
    result = result.loc[result['stock_id'] == stock_id]
    result = result.drop(['stock_id'], axis = 1)
    return result

def generate_data(stock_id, train_or_test):
    result = pd.merge(target(stock_id, train_or_test), book_predictors(stock_id, train_or_test), on='time_id', how='left')
    return result

def generate_train_and_test(stock_id):
    train = generate_data(stock_id, 'train')
    test = generate_data(stock_id, 'test')

    X_train = train.drop(['target', 'time_id'], axis = 1)
    X_test = test.drop(['target', 'time_id'], axis = 1)

    y_train = train['target']
    y_test = test['target']

    return X_train, X_test, y_train, y_test

In [ ]:
def mspe(y_true, y_pred):
    return  (np.mean(np.square((y_true - y_pred) / y_true)))

from sklearn.linear_model import LinearRegression
total_size = 0
total = 0
score_list = []

for i in range(127):
    if not isValidStock(i):
        continue

    #print(i)

    X_train, X_test, y_train, y_test = generate_train_and_test(i)

    reg = LinearRegression().fit(X_train, y_train)
    y_pred = reg.predict(X_test)
    total += mspe(y_test, y_pred) * X_test.shape[0]
    total_size += X_test.shape[0]
    R_2 = reg.score(X_train, y_train)
    score_list.append(R_2)



total_RMSPE = np.sqrt(total / total_size)
total_RMSPE
    




In [ ]:
X_train, X_test, y_train, y_test = generate_train_and_test(1)
X_train
#X_train[X_train.isna().any(axis=1)]

In [ ]:
R2 = np.mean(score_list)

In [ ]:
print(f'Performance of the naive prediction: R2 score: {round(R2,2)}, RMSPE: {round(total_RMSPE,2)}')

In [ ]:
fig = px.scatter(vol_id_0_df_shifted, x='Realized Volatility', y='Target Volatility', title='Target Volatility randomized vs Realized Volatility Last 10 min')
fig.update_yaxes(nticks=20)
fig.update_xaxes(nticks=20)
fig.update_layout(xaxis_range=[0, 0.03])

fig.show()

In [ ]:
vol_id_0_df_shifted.corr().iloc[0,1]

In [ ]:
vol_id_0_df_shifted = vol_id_0_df.copy()
vol_id_0_df_shifted["Realized Volatility"] = vol_id_0_df_shifted["Realized Volatility"].shift(1)
vol_id_0_df_shifted.corr()
#Clearly future time series data has no correlation with previous time series data... Only the prveious 10 min and next 10 min are correlaated. Past data has no effect on future 

In [ ]:
book_train_stock_id_0[book_train_stock_id_0["seconds_in_bucket"] >= 300]
target_train_stock_id_0

In [ ]:
#Checking if previous 5 min vol is better correlated to 10min vol!!!
#Is previous 10min volatility related to next 10 min vol???!!!

book_train_stock_id_0_5min = book_train_stock_id_0[book_train_stock_id_0["seconds_in_bucket"] >= 300]
realized_vol_id_0_5min = book_train_stock_id_0_5min.groupby("time_id")["log_return"].apply(realized_volatility)
realized_vol_id_0_5min
vol_id_0_df_5min = pd.DataFrame(columns=[target_train_stock_id_0["target"], realized_vol_id_0_5min])
realized_vol_id_0_5min
#target_train_stock_id_0["target"]
vol_id_0_df_5min = vol_id_0_df_5min.T
vol_id_0_df_5min
vol_id_0_df_5min.reset_index(inplace=True)
vol_id_0_df_5min = vol_id_0_df_5min.rename(columns={"target": "Target Volatility", "log_return": "Realized Volatility Last 5 min"})
vol_id_0_df_5min

In [ ]:
fig = px.scatter(vol_id_0_df_5min, x='Realized Volatility Last 5 min', y='Target Volatility', title='Target Volatility vs Realized Volatility Last 5 min')
fig.update_yaxes(nticks=20)
fig.update_xaxes(nticks=20)
fig.update_layout(xaxis_range=[0, 0.03])

fig.show()

In [ ]:
vol_id_0_df_5min.corr()

In [ ]:
#Checking if previous 3 min vol is better correlated to 10min vol!!!
#Is previous 10min volatility related to next 10 min vol???!!!

book_train_stock_id_0_3min = book_train_stock_id_0[book_train_stock_id_0["seconds_in_bucket"] >= 300]
realized_vol_id_0_3min = book_train_stock_id_0_3min.groupby("time_id")["log_return"].apply(realized_volatility)
realized_vol_id_0_3min
vol_id_0_df_3min = pd.DataFrame(columns=[target_train_stock_id_0["target"], realized_vol_id_0_3min])
realized_vol_id_0_3min
#target_train_stock_id_0["target"]
vol_id_0_df_3min = vol_id_0_df_3min.T
vol_id_0_df_3min
vol_id_0_df_3min.reset_index(inplace=True)
vol_id_0_df_3min = vol_id_0_df_3min.rename(columns={"target": "Target Volatility", "log_return": "Realized Volatility Last 3 min"})
vol_id_0_df_3min

In [ ]:
fig = px.scatter(vol_id_0_df_5min, x='Realized Volatility Last 5 min', y='Target Volatility', title='Target Volatility vs Realized Volatility Last 3 min')
fig.update_yaxes(nticks=20)
fig.update_xaxes(nticks=20)
fig.update_layout(xaxis_range=[0, 0.03])

fig.show()

In [ ]:
vol_id_0_df_3min.corr()